# Manipulating data
>  In this chapter, you'll learn about the pyspark.sql module, which provides optimized data queries to your Spark session.

- toc: true 
- badges: true
- comments: true
- author: Lucas Nunes
- categories: [Datacamp]
- image: images/datacamp/___

> Note: This is a summary of the course's chapter 2 exercises "Introduction to PySpark" at datacamp. <br>[Github repo](https://github.com/lnunesAI/Datacamp/) / [Course link](https://www.datacamp.com/tracks/machine-learning-scientist-with-python)

In [1]:
%%capture
!pip install pyspark
import pyspark
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession

## Creating columns

<div class=""><p>In this chapter, you'll learn how to use the methods defined by Spark's <code>DataFrame</code> class to perform common data operations.</p>
<p>Let's look at performing column-wise operations. In Spark you can do this using the <code>.withColumn()</code> method, which takes two arguments. First, a string with the name of your new column, and second the new column itself.</p>
<p>The new column must be an object of class <code>Column</code>. Creating one of these is as easy as extracting a column from your DataFrame using <code>df.colName</code>.</p>
<p>Updating a Spark DataFrame is somewhat different than working in <code>pandas</code> because the Spark DataFrame is <em>immutable</em>. This means that it can't be changed, and so columns can't be updated in place.</p>
<p>Thus, all these methods return a new DataFrame. To overwrite the original DataFrame you must reassign the returned DataFrame using the method like so:</p>
<pre><code>df = df.withColumn("newCol", df.oldCol + 1)
</code></pre>
<p>The above code creates a DataFrame with the same columns as <code>df</code> plus a new column, <code>newCol</code>, where every entry is equal to the corresponding entry from <code>oldCol</code>, plus one.</p>
<p>To overwrite an existing column, just pass the name of the column as the first argument!</p>
<p>Remember, a <code>SparkSession</code> called <code>spark</code> is already in your workspace.</p></div>

In [2]:
%%capture
!wget https://github.com/lnunesAI/Datacamp/raw/main/2-machine-learning-scientist-with-python/21-introduction-to-pyspark/datasets/flights_small.csv
file_path = "flights_small.csv"
spark = (SparkSession
  .builder
  .appName("flights")
  .getOrCreate())
flights = (spark.read.format("csv")
  .option("inferSchema", "true")
  .option("header", "true")
  .load(file_path))
flights.createOrReplaceTempView("flights")

Instructions
<ul>
<li>Use the <code>spark.table()</code> method with the argument <code>"flights"</code> to create a DataFrame containing the values of the <code>flights</code> table in the <code>.catalog</code>. Save it as <code>flights</code>.</li>
<li>Show the head of <code>flights</code> using <code>flights.show()</code>. The column <code>air_time</code> contains the duration of the flight in minutes.</li>
<li>Update <code>flights</code> to include a new column called <code>duration_hrs</code>, that contains the duration of each flight in hours.</li>
</ul>

In [3]:
# Create the DataFrame flights
flights = spark.table('flights')

# Show the head
flights.show()

# Add duration_hrs
flights = flights.withColumn('duration_hrs', flights.air_time / 60)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

## SQL in a nutshell

<div class=""><p>As you move forward, it will help to have a basic understanding of SQL. A more in depth look can be found <a href="https://www.datacamp.com/courses/intro-to-sql-for-data-science" target="_blank" rel="noopener noreferrer">here</a>.</p>
<p>A SQL query returns a table derived from one or more tables contained in a database.</p>
<p>Every SQL query is made up of commands that tell the database what you want to do with the data. The two commands that every query has to contain are <code>SELECT</code> and <code>FROM</code>.</p>
<p>The <code>SELECT</code> command is followed by the <em>columns</em> you want in the resulting table.</p>
<p>The <code>FROM</code> command is followed by the name of the table that contains those columns. The minimal SQL query is:</p>
<pre><code>SELECT * FROM my_table;
</code></pre>
<p>The <code>*</code> selects all columns, so this returns the entire table named <code>my_table</code>.</p>
<p>Similar to <code>.withColumn()</code>, you can do column-wise computations within a <code>SELECT</code> statement. For example,</p>
<pre><code>SELECT origin, dest, air_time / 60 FROM flights;
</code></pre>
<p>returns a table with the origin, destination, and duration in hours for each flight.</p>
<p>Another commonly used command is <code>WHERE</code>. This command filters the rows of the table based on some logical condition you specify. The resulting table contains the rows where your condition is true. For example, if you had a table of students and grades you could do:</p>
<pre><code>SELECT * FROM students
WHERE grade = 'A';
</code></pre>
<p>to select all the columns and the rows containing information about students who got As.</p>
<p></p><hr>
Which of the following queries returns a table of tail numbers and destinations for flights that lasted more than 10 hours?<p></p></div>

<pre>
Possible Answers
SELECT dest, tail_num FROM flights WHERE air_time > 10;
<b>SELECT dest, tail_num FROM flights WHERE air_time > 600;</b>
SELECT * FROM flights WHERE air_time > 600;
</pre>

## SQL in a nutshell (2)

<div class=""><p>Another common database task is aggregation. That is, reducing your data by breaking it into chunks and summarizing each chunk.</p>
<p>This is done in SQL using the <code>GROUP BY</code> command. This command breaks your data into groups and applies a function from your <code>SELECT</code> statement to each group.</p>
<p>For example, if you wanted to count the number of flights from each of two origin destinations, you could use the query</p>
<pre><code>SELECT COUNT(*) FROM flights
GROUP BY origin;
</code></pre>
<p><code>GROUP BY origin</code> tells SQL that you want the output to have a row for each unique value of the <code>origin</code> column. The <code>SELECT</code> statement selects the values you want to populate each of the columns. Here, we want to <code>COUNT()</code> every row in each of the groups.</p>
<p>It's possible to <code>GROUP BY</code> more than one column. When you do this, the resulting table has a row for every combination of the unique values in each column. The following query counts the number of flights from SEA and PDX to every destination airport:</p>
<pre><code>SELECT origin, dest, COUNT(*) FROM flights
GROUP BY origin, dest;
</code></pre>
<p>The output will have a row for every combination of the values in <code>origin</code> and <code>dest</code> (i.e. a row listing each origin and destination that a flight flew to). There will also be a column with the <code>COUNT()</code> of all the rows in each group.</p>
<p>Remember, a more in depth look at SQL can be found <a href="https://www.datacamp.com/courses/intro-to-sql-for-data-science" target="_blank" rel="noopener noreferrer">here</a>.</p>
<hr>
<p>What information would this query get? Remember the <code>flights</code> table holds information about flights that departed PDX and SEA in 2014 and 2015. Note that <code>AVG()</code> function gets the average value of a column!</p>
<pre><code>SELECT AVG(air_time) / 60 FROM flights
GROUP BY origin, carrier;
</code></pre></div>

<pre>
Possible Answers
<b>The average length of each airline's flights from SEA and from PDX in hours.</b>
The average length of each flight.
The average length of each airline's flights.
</pre>

## Filtering Data

<p>Now that you have a bit of SQL know-how under your belt, it's easier to talk about the analogous operations using Spark DataFrames.</p>
<p>Let's take a look at the <code>.filter()</code> method. As you might suspect, this is the Spark counterpart of SQL's <code>WHERE</code> clause. The <code>.filter()</code> method takes either an expression that would follow the <code>WHERE</code> clause of a SQL expression as a string, or a Spark Column of boolean (<code>True</code>/<code>False</code>) values.</p>
<p>For example, the following two expressions will produce the same output:</p>
<pre><code>flights.filter("air_time &gt; 120").show()
flights.filter(flights.air_time &gt; 120).show()
</code></pre>
<p>Notice that in the first case, we pass a <em>string</em> to <code>.filter()</code>. In SQL, we would write this filtering task as <code>SELECT * FROM flights WHERE air_time &gt; 120</code>. Spark's <code>.filter()</code> can accept any expression that could go in the <code>WHERE</code>clause of a SQL query (in this case, <code>"air_time &gt; 120"</code>), as long as it is passed as a string.
Notice that in this case, we do not reference the name of the table in the string -- as we wouldn't in the SQL request.</p>
<p>In the second case, we actually pass a <em>column of boolean values</em> to <code>.filter()</code>. Remember that <code>flights.air_time &gt; 120</code> returns a column of boolean values that has <code>True</code> in place of those records in <code>flights.air_time</code> that are over 120, and <code>False</code> otherwise.  </p>
<p>Remember, a <code>SparkSession</code> called <code>spark</code> is already in your workspace, along with the Spark DataFrame <code>flights</code>.</p></div></div>

Instructions
<ul>
<li>Use the <code>.filter()</code> method to find all the flights that flew over 1000 miles two ways:<ul>
<li>First, pass a SQL <strong>string</strong> to <code>.filter()</code> that checks whether the distance is greater than 1000. Save this as <code>long_flights1</code>.</li>
<li>Then pass a column of boolean values to <code>.filter()</code> that checks the same thing. Save this as <code>long_flights2</code>.</li></ul></li>
<li>Use <code>.show()</code> to print heads of both DataFrames and make sure they're actually equal!</li>
</ul>

In [7]:
#shape
(flights.count(), len(flights.columns))

(10000, 17)

In [4]:
# Filter flights by passing a string
long_flights1 = flights.filter("distance > 1000")

# Filter flights by passing a column of boolean values
long_flights2 = flights.filter(flights.distance > 1000)

# Print the data to check they're equal
long_flights1.show()
long_flights2.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|      duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|               6.0|
|2014|    4| 19|    1236|       -4|    1508|       -7|     AS| N309AS|   490|   SEA| SAN|     135|    1050|  12|    36|              2.25|
|2014|   11| 19|    1812|       -3|    2352|       -4|     AS| N564AS|    26|   SEA| ORD|     198|    1721|  18|    12|               3.3|
|2014|    8|  3|    1120|        0|    1415|        2|     AS| N305AS|   656|   SEA| PHX|     154|    1107|  11|    20| 2.566666666666667|
|2014|   11| 12|    2346|  

In [10]:
#shape
(long_flights1.count(), len(long_flights1.columns))

(4883, 17)

## Selecting

<div class=""><p>The Spark variant of SQL's <code>SELECT</code> is the <code>.select()</code> method. This method takes multiple arguments - one for each column you want to select. These arguments can either be the column name as a string (one for each column) or a column object (using the <code>df.colName</code> syntax). When you pass a column object, you can perform operations like addition or subtraction on the column to change the data contained in it, much like inside <code>.withColumn()</code>.</p>
<p>The difference between <code>.select()</code> and <code>.withColumn()</code> methods is that <code>.select()</code> returns only the columns you specify, while <code>.withColumn()</code> returns all the columns of the DataFrame in addition to the one you defined. It's often a good idea to drop columns you don't need at the beginning of an operation so that you're not dragging around extra data as you're wrangling. In this case, you would use <code>.select()</code> and not <code>.withColumn()</code>.</p>
<p>Remember, a SparkSession called <code>spark</code> is already in your workspace, along with the Spark DataFrame <code>flights</code>.</p></div>

Instructions
<ul>
<li>Select the columns <code>tailnum</code>, <code>origin</code>, and <code>dest</code> from <code>flights</code> by passing the column names as strings. Save this as <code>selected1</code>.</li>
<li>Select the columns <code>origin</code>, <code>dest</code>, and <code>carrier</code> using the <code>df.colName</code> syntax and then filter the result using both of the filters already defined for you (<code>filterA</code> and <code>filterB</code>) to only keep flights from SEA to PDX. Save this as <code>selected2</code>.</li>
</ul>

In [12]:
# Select the first set of columns
selected1 = flights.select('tailnum', 'origin', 'dest')

# Select the second set of columns
temp = flights.select(flights.origin, flights.dest, flights.carrier)

# Define first filter
filterA = flights.origin == "SEA"

# Define second filter
filterB = flights.dest == "PDX"

# Filter the data, first by filterA then by filterB
selected2 = temp.filter(filterA).filter(filterB)

## Selecting II

<div class=""><p>Similar to SQL, you can also use the <code>.select()</code> method to perform column-wise operations. When you're selecting a column using the <code>df.colName</code> notation, you can perform any column operation and the <code>.select()</code> method will return the transformed column. For example,</p>
<pre><code>flights.select(flights.air_time/60)
</code></pre>
<p>returns a column of flight durations in hours instead of minutes. You can also use the <code>.alias()</code> method to rename a column you're selecting. So if you wanted to <code>.select()</code> the column <code>duration_hrs</code> (which isn't in your DataFrame) you could do</p>
<pre><code>flights.select((flights.air_time/60).alias("duration_hrs"))
</code></pre>
<p>The equivalent Spark DataFrame method <code>.selectExpr()</code> takes SQL expressions as a string:</p>
<pre><code>flights.selectExpr("air_time/60 as duration_hrs")
</code></pre>
<p>with the SQL <code>as</code> keyword being equivalent to the <code>.alias()</code> method. To select multiple columns, you can pass multiple strings.</p>
<p>Remember, a <code>SparkSession</code> called <code>spark</code> is already in your workspace, along with the Spark DataFrame <code>flights</code>.</p></div>

Instructions
<div class="exercise--instructions__content"><p>Create a table of the average speed of each flight both ways.</p>
<ul>
<li>Calculate average speed by dividing the <code>distance</code> by the <code>air_time</code> (converted to hours). Use the <code>.alias()</code> method name this column <code>"avg_speed"</code>. Save the output as the variable <code>avg_speed</code>.</li>
<li>Select the columns <code>"origin", "dest", "tailnum"</code>, and <code>avg_speed</code> (without quotes!). Save this as <code>speed1</code>.</li>
<li>Create the same table using <code>.selectExpr()</code> and a string containing a SQL expression. Save this as <code>speed2</code>.</li>
</ul></div>

In [19]:
# Define avg_speed
avg_speed = (flights.distance/(flights.air_time/60)).alias("avg_speed")

# Select the correct columns
speed1 = flights.select("origin", "dest", "tailnum", avg_speed)

# Create the same table using a SQL expression
speed2 = flights.selectExpr("origin", "dest", "tailnum", "distance/(air_time/60) as avg_speed")

In [21]:
speed1.show()

+------+----+-------+------------------+
|origin|dest|tailnum|         avg_speed|
+------+----+-------+------------------+
|   SEA| LAX| N846VA| 433.6363636363636|
|   SEA| HNL| N559AS| 446.1666666666667|
|   SEA| SFO| N847VA|367.02702702702703|
|   PDX| SJC| N360SW| 411.3253012048193|
|   SEA| BUR| N612AS| 442.6771653543307|
|   PDX| DEN| N646SW|491.40495867768595|
|   PDX| OAK| N422WN|             362.0|
|   SEA| SFO| N361VA| 415.7142857142857|
|   SEA| SAN| N309AS| 466.6666666666667|
|   SEA| ORD| N564AS| 521.5151515151515|
|   SEA| LAX| N323AS| 440.3076923076923|
|   SEA| PHX| N305AS|431.29870129870125|
|   SEA| LAS| N433AS| 409.6062992125984|
|   SEA| ANC| N765AS|474.75409836065575|
|   SEA| SFO| N713AS| 315.8139534883721|
|   PDX| SFO| N27205| 366.6666666666667|
|   SEA| SMF| N626AS|477.63157894736844|
|   SEA| MDW| N8634A|481.38888888888886|
|   SEA| BOS| N597AS| 516.4137931034483|
|   PDX| BUR| N215AG| 441.6216216216216|
+------+----+-------+------------------+
only showing top

## Aggregating

<div class=""><p>All of the common aggregation methods, like <code>.min()</code>, <code>.max()</code>, and <code>.count()</code> are <code>GroupedData</code> methods. These are created by calling the <code>.groupBy()</code> DataFrame method. You'll learn exactly what that means in a few exercises. For now, all you have to do to use these functions is call that method on your DataFrame. For example, to find the minimum value of a column, <code>col</code>, in a DataFrame, <code>df</code>, you could do</p>
<pre><code>df.groupBy().min("col").show()
</code></pre>
<p>This creates a <code>GroupedData</code> object (so you can use the <code>.min()</code> method), then finds the minimum value in <code>col</code>, and returns it as a DataFrame.</p>
<p>Now you're ready to do some aggregating of your own!</p>
<p>A <code>SparkSession</code> called <code>spark</code> is already in your workspace, along with the Spark DataFrame <code>flights</code>.</p></div>

In [19]:
from pyspark.sql.types import IntegerType
flights = flights.withColumn("air_time", flights["air_time"].cast(IntegerType()))
flights = flights.withColumn("dep_delay", flights["dep_delay"].cast(IntegerType()))



---



Instructions
<ul>
<li>Find the length of the shortest (in terms of distance) flight that left PDX by first <code>.filter()</code>ing and using the <code>.min()</code> method. Perform the filtering by referencing the column directly, not passing a SQL string.</li>
<li>Find the length of the longest (in terms of time) flight that left SEA by <code>filter()</code>ing and using the <code>.max()</code> method. Perform the filtering by referencing the column directly, not passing a SQL string.</li>
</ul>

In [20]:
# Find the shortest flight from PDX in terms of distance
flights.filter(flights.origin == 'PDX').groupBy().min("distance").show()

# Find the longest flight from SEA in terms of air time
flights.filter(flights.origin == 'SEA').groupBy().max("air_time").show()

+-------------+
|min(distance)|
+-------------+
|          106|
+-------------+

+-------------+
|max(air_time)|
+-------------+
|          409|
+-------------+



## Aggregating II

<div class=""><p>To get you familiar with more of the built in aggregation methods, here's a few more exercises involving the <code>flights</code> table!</p>
<p>Remember, a <code>SparkSession</code> called <code>spark</code> is already in your workspace, along with the Spark DataFrame <code>flights</code>.</p></div>

Instructions
<ul>
<li>Use the <code>.avg()</code> method to get the average air time of Delta Airlines flights (where the <code>carrier</code> column has the value <code>"DL"</code>) that left SEA. The place of departure is stored in the column <code>origin</code>. <code>show()</code> the result.</li>
<li>Use the <code>.sum()</code> method to get the total number of hours all planes in this dataset spent in the air by creating a column called <code>duration_hrs</code> from the column <code>air_time</code>. <code>show()</code> the result.</li>
</ul>

In [21]:
# Average duration of Delta flights
flights.filter(flights.origin == 'SEA').filter(flights.carrier == 'DL').groupBy().avg('air_time').show()

# Total hours in the air
flights.withColumn("duration_hrs", flights.air_time/60).groupBy().sum('duration_hrs').show()

+------------------+
|     avg(air_time)|
+------------------+
|188.20689655172413|
+------------------+

+------------------+
| sum(duration_hrs)|
+------------------+
|25289.600000000126|
+------------------+



## Grouping and Aggregating I

<div class=""><p>Part of what makes aggregating so powerful is the addition of groups. PySpark has a whole class devoted to grouped data frames: <code>pyspark.sql.GroupedData</code>, which you saw in the last two exercises.</p>
<p>You've learned how to create a grouped DataFrame by calling the <code>.groupBy()</code> method on a DataFrame with no arguments.</p>
<p>Now you'll see that when you pass the name of one or more columns in your DataFrame to the <code>.groupBy()</code> method, the aggregation methods behave like when you use a <code>GROUP BY</code> statement in a SQL query!</p>
<p>Remember, a <code>SparkSession</code> called <code>spark</code> is already in your workspace, along with the Spark DataFrame <code>flights</code>.</p></div>

Instructions
<ul>
<li>Create a DataFrame called <code>by_plane</code> that is grouped by the column <code>tailnum</code>.</li>
<li>Use the <code>.count()</code> method with no arguments to count the number of flights each plane made.</li>
<li>Create a DataFrame called <code>by_origin</code> that is grouped by the column <code>origin</code>.</li>
<li>Find the <code>.avg()</code> of the <code>air_time</code> column to find average duration of flights from PDX and SEA.</li>
</ul>

In [22]:
# Group by tailnum
by_plane = flights.groupBy("tailnum")

# Number of flights each plane made
by_plane.count().show()

# Group by origin
by_origin = flights.groupBy("origin")

# Average duration of flights from PDX and SEA
by_origin.avg("air_time").show()

+-------+-----+
|tailnum|count|
+-------+-----+
| N442AS|   38|
| N102UW|    2|
| N36472|    4|
| N38451|    4|
| N73283|    4|
| N513UA|    2|
| N954WN|    5|
| N388DA|    3|
| N567AA|    1|
| N516UA|    2|
| N927DN|    1|
| N8322X|    1|
| N466SW|    1|
|  N6700|    1|
| N607AS|   45|
| N622SW|    4|
| N584AS|   31|
| N914WN|    4|
| N654AW|    2|
| N336NW|    1|
+-------+-----+
only showing top 20 rows

+------+------------------+
|origin|     avg(air_time)|
+------+------------------+
|   SEA| 160.4361496051259|
|   PDX|137.11543248288737|
+------+------------------+



## Grouping and Aggregating II

<div class=""><p>In addition to the <code>GroupedData</code> methods you've already seen, there is also the <code>.agg()</code> method.
This method lets you pass an aggregate column expression that uses any of the aggregate functions from the <code>pyspark.sql.functions</code> submodule.</p>
<p>This submodule contains many useful functions for computing things like standard deviations. All the aggregation functions in this submodule take the name of a column in a <code>GroupedData</code> table.</p>
<p>Remember, a <code>SparkSession</code> called <code>spark</code> is already in your workspace, along with the Spark DataFrame <code>flights</code>. The grouped DataFrames you created in the last exercise are also in your workspace.</p></div>

Instructions
<ul>
<li>Import the submodule <code>pyspark.sql.functions</code> as <code>F</code>.</li>
<li>Create a <code>GroupedData</code> table called <code>by_month_dest</code> that's grouped by both the <code>month</code> and <code>dest</code> columns. Refer to the two columns by passing both strings as separate arguments.</li>
<li>Use the <code>.avg()</code> method on the <code>by_month_dest</code> DataFrame to get the average <code>dep_delay</code> in each month for each destination.</li>
<li>Find the standard deviation of <code>dep_delay</code> by using the <code>.agg()</code> method with the function <code>F.stddev()</code>.</li>
</ul>

In [23]:
# Import pyspark.sql.functions as F
import pyspark.sql.functions as F

# Group by month and dest
by_month_dest = flights.groupBy('month', 'dest')

# Average departure delay by month and destination
by_month_dest.avg('dep_delay').show()

# Standard deviation of departure delay
by_month_dest.agg(F.stddev('dep_delay')).show()

+-----+----+-------------------+
|month|dest|     avg(dep_delay)|
+-----+----+-------------------+
|    4| PHX| 1.6833333333333333|
|    1| RDM|             -1.625|
|    5| ONT| 3.5555555555555554|
|    7| OMA|               -6.5|
|    8| MDW|               7.45|
|    6| DEN|  5.418181818181818|
|    5| IAD|               -4.0|
|   12| COS|               -1.0|
|   11| ANC|  7.529411764705882|
|    5| AUS|              -0.75|
|    5| COS| 11.666666666666666|
|    2| PSP|                0.6|
|    4| ORD|0.14285714285714285|
|   10| DFW| 18.176470588235293|
|   10| DCA|               -1.5|
|    8| JNU|             18.125|
|   11| KOA|               -1.0|
|   10| OMA|-0.6666666666666666|
|    6| ONT|              9.625|
|    3| MSP|                3.2|
+-----+----+-------------------+
only showing top 20 rows

+-----+----+----------------------+
|month|dest|stddev_samp(dep_delay)|
+-----+----+----------------------+
|    4| PHX|    15.003380033491737|
|    1| RDM|     8.830749846821778|
| 

## Joining

<div class=""><p>Another very common data operation is the <em>join</em>. Joins are a whole topic unto themselves, so in this course we'll just look at simple joins. If you'd like to learn more about joins, you can take a look <a href="https://www.datacamp.com/courses/merging-dataframes-with-pandas" target="_blank" rel="noopener noreferrer">here</a>.</p>
<p>A join will combine two different tables along a column that they share. This column is called the <em>key</em>. Examples of keys here include the <code>tailnum</code> and <code>carrier</code> columns from the <code>flights</code> table.</p>
<p>For example, suppose that you want to know more information about the plane that flew a flight than just the tail number. This information isn't in the <code>flights</code> table because the same plane flies many different flights over the course of two years, so including this information in every row would result in a lot of duplication. To avoid this, you'd have a second table that has only one row for each plane and whose columns list all the information about the plane, including its tail number. You could call this table <code>planes</code></p>
<p>When you join the <code>flights</code> table to this table of airplane information, you're adding all the columns from the <code>planes</code> table to the <code>flights</code> table. To fill these columns with information, you'll look at the tail number from the <code>flights</code> table and find the matching one in the <code>planes</code> table, and then use that row to fill out all the new columns.</p>
<p>Now you'll have a much bigger table than before, but now every row has all information about the plane that flew that flight!</p>
<hr>
<p>Which of the following is <strong>not</strong> true?</p></div>

<pre>
Possible Answers
Joins combine tables.
Joins add information to a table.
Storing information in separate tables can reduce repetition.
<b>There is only one kind of join.</b>
</pre>

**If there were only one kind of join, it would be tough to create some more complicated kinds of tables.**

## Joining II

<div class=""><p>In PySpark, joins are performed using the DataFrame method <code>.join()</code>. This method takes three arguments. The first is the second DataFrame that you want to join with the first one. The second argument, <code>on</code>, is the name of the key column(s) as a string. The names of the key column(s) must be the same in each table. The third argument, <code>how</code>, specifies the kind of join to perform. In this course we'll always use the value <code>how="leftouter"</code>.</p>
<p>The <code>flights</code> dataset and a new dataset called <code>airports</code> are already in your workspace.</p></div>

In [32]:
%%capture
!wget https://github.com/lnunesAI/Datacamp/raw/main/2-machine-learning-scientist-with-python/21-introduction-to-pyspark/datasets/airports.csv
airports = (spark.read.format("csv")
  .option("inferSchema", "true")
  .option("header", "true")
  .load('airports.csv'))
airports.createOrReplaceTempView("airports")

Instructions
<ul>
<li>Examine the <code>airports</code> DataFrame by calling <code>.show()</code>. Note which key column will let you join <code>airports</code> to the <code>flights</code> table.</li>
<li>Rename the <code>faa</code> column in <code>airports</code> to <code>dest</code> by re-assigning the result of <code>airports.withColumnRenamed("faa", "dest")</code> to <code>airports</code>.</li>
<li>Join the <code>flights</code> with the <code>airports</code> DataFrame on the <code>dest</code> column by calling the <code>.join()</code> method on <code>flights</code>. Save the result as <code>flights_with_airports</code>.<ul>
<li>The first argument should be the other DataFrame, <code>airports</code>.</li>
<li>The argument <code>on</code> should be the key column.</li>
<li>The argument <code>how</code> should be <code>"leftouter"</code>.</li></ul></li>
<li>Call <code>.show()</code> on <code>flights_with_airports</code> to examine the data again. Note the new information that has been added.</li>
</ul>

In [36]:
# Examine the data
print(airports.show())

# Rename the faa column
airports = airports.withColumnRenamed("faa", "dest")

# Join the DataFrames
flights_with_airports = flights.join(airports, on='dest', how="leftouter")

# Examine the new DataFrame
print(flights_with_airports.show())

+---+--------------------+----------------+-----------------+----+---+---+
|faa|                name|             lat|              lon| alt| tz|dst|
+---+--------------------+----------------+-----------------+----+---+---+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.7812318| 492| -5|  A|
|0P2|Shoestring Aviati...|      39.7948244|      -76.6471914|1000| -5|  U|
|0S9|Jefferson County ...|      48.0538086|     -122.8106436| 108| -8|  A|
|0W3|Harford County Ai...